## Pick last timestep of every run of multirun
* This is me doing the same thing I already did in `CompareMultipleRuns.ipynb` but in different approach because I like wasting time

In [51]:
%matplotlib widget

In [52]:
import xarray as xr
import hvplot.xarray
import pyvista as pv
from JulesD3D.processNetCDF import fixCORs, fixMeshGrid
from JulesD3D.utils import quickDF, folderPicker
from os import path, walk
import ipywidgets as widgets
from IPython.display import Markdown as md

In [53]:
parent_folder = '/Users/julesblom/ThesisResults/'
scenario_folder = folderPicker(parent_folder)
scenario_folder

Select(description='Scenario folder:', layout=Layout(height='120px', width='100%'), options=('/Users/julesblom…

## ⚠️ Things blow up when you include Run01 in this folder  ¯\_(ツ)_/¯   ⚠️

In [57]:
all_nc_files_in_folder = '/Users/julesblom/ThesisResults/Slope0.85/*.nc'
scenario = scenario_folder.value[-10:-1]
scenario

'Slope0.85'

In [61]:
# hack because first dataset is composed slightly differnt to restart datasets 
first_ncfilename = '/Users/julesblom/ThesisResults/Slope085_Run01.nc'
# '/Users/julesblom/ThesisResults/Slope125_Run01.nc'

first = xr.open_dataset(first_ncfilename)

In [62]:
combined = xr.open_mfdataset(all_nc_files_in_folder, combine='nested', concat_dim="time",# chunks={'time': 5},
                             data_vars='minimal', coords='minimal', join='left', compat='override')

In [63]:
combined = fixCORs(combined)
combined = fixMeshGrid(combined, mystery_flag=True)
combined['bottom_diff'] = first.DPS.isel(time = 0) - combined.DPS # Make accumulated deposition/erosion DataArray

------ Fixing mesh grid, assuming a uniform grid ------
x_gridstep:	 200.0 m
y_gridstep:	 200.0 m
width:		 26000.0 m
length:		 36200.0 m
132 x 182 grid


In [65]:
_, _, files = next(walk(results_folder[:-4]))
nr_of_datasets = len(files) - 1 # .DS_Store wrecking things here
nr_time_steps = int(combined.time.size)
steps_per_dataset = int(nr_time_steps/nr_of_datasets)
times = [i for i in range(steps_per_dataset-1, nr_time_steps+1, steps_per_dataset)]

In [66]:
md(f"""
# Times
| Name | Value |
| --- | --- |
| Nr of timesteps total | {nr_time_steps} |
| Nr of datasets | {nr_of_datasets} |
| Timestep per set | {steps_per_dataset} |
""")


# Times
| Name | Value |
| --- | --- |
| Nr of timesteps total | 144 |
| Nr of datasets | 12 |
| Timestep per set | 12 |


In [67]:
times

[11, 23, 35, 47, 59, 71, 83, 95, 107, 119, 131, 143]

In [69]:
# Returns a new dataset with each array indexed along every n-th value for the specified dimension(s)
# So only timesteps on last of each run
# Ignore channel, only basin part
only_last_timestep_each_run = combined.thin({'time': steps_per_dataset }).isel(N=slice(75,None))

# Depocenter coords
Ignoring channel

In [70]:
only_last_timestep_each_run.time

<xarray.DataArray 'time' (time: 12)>
array(['2020-02-01T20:30:00.000000000', '2020-02-04T10:00:00.000000000',
       '2020-02-05T06:30:00.000000000', '2020-02-08T16:30:00.000000000',
       '2020-02-12T23:00:00.000000000', '2020-02-17T05:30:00.000000000',
       '2020-02-21T12:00:00.000000000', '2020-02-25T18:30:00.000000000',
       '2020-03-01T01:00:00.000000000', '2020-03-05T07:30:00.000000000',
       '2020-03-09T14:00:00.000000000', '2020-03-13T20:30:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-02-01T20:30:00 ... 2020-03-13T20:30:00
Attributes:
    standard_name:  time
    long_name:      time

# Plan (cumulative thickest point)
* Include first run, very important! Bottom diff should be relative to first run
* use thinned dataset
* loop over each time in thinned datasat (should be last of each 5th run)
* get the depocenter coords

In [71]:
thickest_points = []
lowest_point = []

for time in range(1, only_last_timestep_per_run.time.size):
    print('time', time)
    bottom_diff_at_time = only_last_timestep_per_run.bottom_diff.isel(time=time).squeeze()
    
    thickest_deposit_height = bottom_diff_at_time.max().values    
    lowest_deposit_height = bottom_diff_at_time.min().values

    print("thickest:", thickest_deposit_height, "lowest", lowest_deposit_height)
    
    x_coord_erosion = bottom_diff_at_time.where(bottom_diff_at_time == lowest_deposit_height, drop=True).squeeze().XZ.values
    y_coord_erosion = bottom_diff_at_time.where(bottom_diff_at_time == lowest_deposit_height, drop=True).squeeze().YZ.values
    
    x_coord_depocenter = bottom_diff_at_time.where(bottom_diff_at_time == thickest_deposit_height, drop=True).squeeze().XZ.values
    y_coord_depocenter = bottom_diff_at_time.where(bottom_diff_at_time == thickest_deposit_height, drop=True).squeeze().YZ.values
    
    thickest_points.append(
        {"x": x_coord_depocenter,
         "y": y_coord_depocenter,
         "height": float(thickest_deposit_height),
         "run": int(time-1*5)
        })
#     lowest_point_coords.append([int(x_coord_erosion),int(y_coord_depocenter) ])

time 1
thickest: 0.1473999 lowest -0.7699585
time 2
thickest: 0.1687622 lowest -0.7974243
time 3
thickest: 0.3076172 lowest -1.4804688
time 4
thickest: 0.4595337 lowest -2.1726685
time 5
thickest: 0.60375977 lowest -2.8484497
time 6
thickest: 0.743103 lowest -3.5142517
time 7
thickest: 0.88323975 lowest -4.1671753
time 8
thickest: 1.0234375 lowest -4.8122253
time 9
thickest: 1.1583862 lowest -5.4520874
time 10
thickest: 1.2886353 lowest -6.087158
time 11
thickest: 1.4125366 lowest -6.724365


In [28]:
thickest_points

[{'x': array([11500, 14500]),
  'y': array([18600, 18600]),
  'height': 0.0380859375,
  'run': -4},
 {'x': array(11500), 'y': array(18600), 'height': 0.1112060546875, 'run': -3},
 {'x': array(11500), 'y': array(18600), 'height': 0.27044677734375, 'run': -2},
 {'x': array(11500), 'y': array(18600), 'height': 0.42083740234375, 'run': -1},
 {'x': array(11500), 'y': array(18600), 'height': 0.561767578125, 'run': 0},
 {'x': array(11700), 'y': array(18800), 'height': 0.6973876953125, 'run': 1},
 {'x': array(12300), 'y': array(19600), 'height': 0.83270263671875, 'run': 2},
 {'x': array(12300), 'y': array(19600), 'height': 0.9676513671875, 'run': 3},
 {'x': array(12300), 'y': array(19600), 'height': 1.10107421875, 'run': 4},
 {'x': array(12300), 'y': array(19600), 'height': 1.232421875, 'run': 5},
 {'x': array(12300), 'y': array(19600), 'height': 1.35595703125, 'run': 6}]

In [13]:
lowest_point_coords

[[13100, 18200],
 [13500, 18400],
 [13500, 18400],
 [13500, 18600],
 [13500, 18600],
 [13500, 18600],
 [13500, 18600],
 [13500, 21600],
 [13500, 21600],
 [13500, 21600]]


# Plot Slope 1.25

In [16]:
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from mpl_toolkits.mplot3d import Axes3D
from JulesD3D.D3DModel import D3DModel
%load_ext autoreload

In [17]:
%reload_ext autoreload

In [19]:
# from JulesD3D.grid import Grid
# from JulesD3D.dep import Depth

# for root, dirs, files in walk(model_folder):
#     for file in files:
#         if file.endswith('grd'):
#             grid_filename = path.join(model_folder, file)
#         if file.endswith('dep'):
#             dep_filename = path.join(model_folder, file)
# grid = Grid.read(grid_filename)
# grid.shape
# dep = Depth.read(dep_filename, grid.shape)
# depth_matrix = dep.values[0:-1,0:-1]

In [60]:
thick_125_x, thick_125_y = zip(*thickest_point_coords)
labels_125 = [f"Run {((i+1)*5)}: x m" for i in range(len(thick_125_x))]
labels_125

['Run 5: x m',
 'Run 10: x m',
 'Run 15: x m',
 'Run 20: x m',
 'Run 25: x m',
 'Run 30: x m',
 'Run 35: x m',
 'Run 40: x m',
 'Run 45: x m',
 'Run 50: x m']

In [30]:
model_folder = "/Users/julesblom/Dropbox/TU/Master/Thesis/Runs/Current/ResetEvery5_Slope1.25/Run01"

In [31]:
slope125 = D3DModel(folderpath=model_folder)

D3D model filenames


,type,filename
0,B.C. Constituents,5050_15.bcc
1,Depth,gen_36_by_26km.dep
2,MDF,36km_200m_W60Channel.mdf
3,Boundary,QChannelZdeep.bnd
4,Grid,36km_26km_W60Channel.grd
5,Enclosure,36km_1cellchannel_15km.enc
6,Morphology,20_20.mor
7,Sediments,SandSilt.sed
8,B.C. Time series,discharge4500.bct


Boundary file


,name,type,forcing,m1,n1,m2,n2,reflection coefficient,vertical profile,label1,label2
0,Discharge,Q,T,66,1,66,1,200.0,3d-profile,NaN,NaN
1,deep,Z,T,2,182,131,182,200.0,NaN,NaN,NaN


Enclosure values


,x,y
0,1,75
1,64,75
2,64,1
3,69,1
4,69,75
5,132,75
6,132,182
7,1,182
8,1,75


In [65]:
fig, ax = slope125.plotMap(enclosure=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [66]:
ax.scatter(thick_125_x, thick_125_y, c='white', edgecolors="black")
for i  in range(0, len(labels_125), 3):
    print(i)
    print(labels_125[i])
    anno = ax.annotate(labels_125[i], (thick_125_x[i], thick_125_y[i]), c='white')
    anno.set_path_effects([PathEffects.withStroke(linewidth=0.4, foreground='black')])


0
Run 5: x m
3
Run 20: x m
6
Run 35: x m
9
Run 50: x m


In [41]:
plt.show()